# Data Analytics Project - Preprocessing

# Generation of Correlated and co-integrated pairs in the stock market

---

# 1. Importing required modules

In [1]:
import os
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import coint

---

# 2. Defining time window to find correlation and co-integration

In [2]:
START_DATE       = '2018-01-07'
END_DATE         = '2019-08-28'
EVAL_PERIOD      = 90 # days
CORR_THRESHOLD   = 0.85
COINT_THRESHOLD  = 0.05
SPREAD_RANGE     = 500.0
SPREAD_RATIO     = 5

---

# 3. Loading dataframe and defining sectors to make stock pairs

## 3.1. Sector tickers file

In [3]:
sector_tickers = pd.read_csv('../Storage/stocks_by_sectors.csv')
sector_tickers.head()

,Symbol,Company,Sector
0,ABB,ABB India Limited,Heavy Electrical Equipment
1,AEGISLOG,AEGIS LOGISTICS LTD.,Oil Marketing & Distribution
2,AMARAJABAT,AMARA RAJA BATTERIES LTD.,Auto Parts & Equipment
3,AMBALALSA,AMBALAL SARABHAI ENTERPRISES LTD.,Pharmaceuticals
4,HDFC,HOUSING DEVELOPMENT FINANCE CORP.LTD.,Housing Finance


## 3.2. Creating the Pairs DataFrame

In [4]:
pairs_df = pd.DataFrame(columns = ['Symbol_1', 'Symbol_2', 'StartDate', 'EndDate', 'Correlation', 'PValue'])
pairs_df

,Symbol_1,Symbol_2,StartDate,EndDate,Correlation,PValue


## 3.3. Defining Sectors of the stock market to make pairs

In [5]:
# sectors = set(sector_tickers['Sector'].to_list())
sectors = ['2/3 Wheelers', 'Commercial Vehicles']
print("Number of sectors:", len(sectors))

Number of sectors: 2


---

# 4. Calculating z-score

In [6]:
def zscore(data):
    return (data - data.mean())/np.std(data)

---

# 5. Creating pairs

In [7]:
# Go through all sectors
for sector in sectors:
    
#     print('------------------------------------------------\n')
#     print(sector+'\n')
#     print('------------------------------------------------\n')
    
    # Getting stocks in a particular sector
    sector_df = sector_tickers[sector_tickers.Sector == sector]
    
#     print(sector_df.head())
    
    # Get list of all stock tickers of a sector
    companies = sector_df['Symbol'].tolist()
    
    # Go through all companies in a sector
    for i in range(len(companies)):
        
        # Chosing first company in pair
        symbol1 = companies[i]
        
        # If data for company not available, move to the next iteration
        if not os.path.isfile('../Storage/Companies_with_names_exchange/' + symbol1 + 'NSE.csv'):
            continue
        
        # Go through all other companies to make pairs
        for j in range(i + 1, len(companies)):
            
            # Chosing second company in pair
            symbol2 = companies[j]
            
            if not os.path.isfile('../Storage/Companies_with_names_exchange/' + symbol2 + 'NSE.csv'):
                continue
            
            # Get dataframes of both csvs
            symbol1_df = pd.read_csv('../Storage/Companies_with_names_exchange/' + symbol1 + 'NSE.csv')
            symbol2_df = pd.read_csv('../Storage/Companies_with_names_exchange/' + symbol2 + 'NSE.csv')
            
            # eliminate all data before '2018-12-01'
            symbol1_df = symbol1_df[symbol1_df.Date >= START_DATE]
            symbol2_df = symbol2_df[symbol2_df.Date >= START_DATE]
            
            # drop columns 'High', 'Low', 'Open', 'Adj Close', 'Volume'
            symbol1_df = symbol1_df.drop(columns=['High', 'Low', 'Open', 'Adj Close', 'Volume'])
            symbol2_df = symbol2_df.drop(columns=['High', 'Low', 'Open', 'Adj Close', 'Volume'])
            
            # rename 'Close' column
            symbol1_df = symbol1_df.rename(columns={'Close': symbol1 + '_Close'})
            symbol2_df = symbol2_df.rename(columns={'Close': symbol2 + '_Close'})
            
            # set_index to Date
            symbol1_df = symbol1_df.set_index(keys=['Date'])
            symbol2_df = symbol2_df.set_index(keys=['Date'])
            
            # drop company and exchange columns
            symbol1_df = symbol1_df.drop(columns=['Company', 'Exchange'])
            symbol2_df = symbol2_df.drop(columns=['Company', 'Exchange'])
            
            if len(symbol1_df) != len(symbol2_df):
                continue
            
            # join both symbols into a signle pair_df
            master_pair_df = symbol1_df.join(symbol2_df)
            
            if (master_pair_df[symbol1+'_Close'].mean() / master_pair_df[symbol2+'_Close'].mean()) >= SPREAD_RATIO:
                continue
            
            # reset_index
            master_pair_df = master_pair_df.reset_index()
            
            del symbol1_df
            del symbol2_df
            
            # Logic:
            #     - start_date = '2019-01-07'
            #     - end_date   = '2020-08-28'
            #         - from start_date fetch 90 days of data (check if start_date + 90 > end_date)
            #         - check correlation and cointegration on the pair
            #         - if it is above the required THRESHOLDS
            #              - calculate the spread & zscore of spread
            #              - store the CSV file
            #     - ADD 15 days to start_date and REPEAT
            
            start_dt = START_DATE
            eval_dt  = str(pd.to_datetime(start_dt) + pd.DateOffset(days=90)).split()[0]
            count = 0
            
            while eval_dt < END_DATE:
                
                pair_df = master_pair_df[master_pair_df['Date'] >= start_dt]
                
                if len(pair_df) < 5:
                    break
                
                pair_df = pair_df[pair_df['Date'] <= eval_dt]
                pair_df = pair_df.dropna()
                
                # f.write(f'Evaluating: {symbol1} - {symbol2}, {start_dt} to {eval_dt}, Num data points - {len(pair_df)}\n')
                
                corr_df = pair_df.corr()
                score, pvalue, _ = coint(pair_df[symbol1 + '_Close'], pair_df[symbol2 + '_Close'])
                corr_value = corr_df.loc[symbol1 + '_Close', symbol2 + '_Close']
                
                if (corr_value > CORR_THRESHOLD) & (pvalue < COINT_THRESHOLD):
                    pair_df["Spread"] = pair_df[symbol1 + "_Close"] - pair_df[symbol2 + "_Close"]
                    spread_mean = abs(pair_df['Spread'].mean())
                
                    if spread_mean < SPREAD_RANGE:
                        # f.write(f'MATCH FOUND: CORR - {corr_value}, COINT - {pvalue}\n')
                        
                        pairs_df = pairs_df.append(
                            {'Symbol_1': symbol1,
                             'Symbol_2': symbol2,
                             'StartDate': start_dt,
                             'EndDate': eval_dt,
                             'Correlation': corr_value,
                             'PValue': pvalue
                            }, ignore_index=True)
                        
                        pair_df["Spread"] = pair_df[symbol1 + "_Close"] - pair_df[symbol2 + "_Close"]
                        pair_df["zscore"] = zscore(pair_df["Spread"])
                        
                        pair_df.to_csv('../Storage/pairs_data/'+symbol1+'-'+symbol2+'-'+str(count)+'.csv', index=False)
                        count += 1
                
                del pair_df
                del corr_df
                
                start_dt = str(pd.to_datetime(start_dt) + pd.DateOffset(days=15)).split()[0]
                eval_dt  = str(pd.to_datetime(start_dt) + pd.DateOffset(days=90)).split()[0]

pairs_df.to_csv('pairs_with_time_range.csv', index=False)


---

# 6. Showing a sample of results

## 6.1. The pairs found to have a high Correlation and low p value

In [9]:
pairs_with_time_range = pd.read_csv('./pairs_with_time_range.csv')
pairs_with_time_range.head()

,Symbol_1,Symbol_2,StartDate,EndDate,Correlation,PValue
0,HEROMOTOCO,BAJAJ-AUTO,2018-09-04,2018-12-03,0.899256,0.024984
1,HEROMOTOCO,BAJAJ-AUTO,2018-10-04,2019-01-02,0.905777,0.013314
2,HEROMOTOCO,BAJAJ-AUTO,2019-05-17,2019-08-15,0.896455,0.047065
3,BEML,ESCORTS,2018-08-05,2018-11-03,0.974214,0.003684
4,BEML,ESCORTS,2019-01-02,2019-04-02,0.925196,0.008177


## 6.2. The pairs data within those ranges

In [10]:
pair1_df = pd.read_csv('../Storage/pairs_data/TATAMOTORS-TATAMTRDVR-5.csv')
pair1_df.head()

,Date,TATAMOTORS_Close,TATAMTRDVR_Close,Spread,zscore
0,2018-09-19,252.600006,136.699997,115.900009,2.473078
1,2018-09-21,250.350006,132.050003,118.300003,2.671696
2,2018-09-24,240.199997,128.050003,112.149994,2.162735
3,2018-09-25,240.800003,127.050003,113.750000,2.295148
4,2018-09-26,233.350006,123.050003,110.300003,2.009634
